## SASRec with Explanation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
from collections import defaultdict
import pandas as pd
from tqdm import tqdm
from itertools import combinations
from numpy.linalg import norm
import os
import zipfile
import requests

In [2]:
CFG = {
    "MAX_LEN": 50,
    "HIDDEN": 128,
    "N_LAYERS": 2,
    "N_HEADS": 4,
    "DROPOUT": 0.2,
    "LR": 3e-4,
    "BATCH_SIZE": 256,
    "EPOCHS": 20,
    "MASK_PROB": 0.15,
    "NEG_SAMPLES": 100,
    "MIN_INTERACTIONS": 5,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu"
}

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

## load_and_prepare
**What happens here?**

- Reads interaction dataset (like MovieLens).
- Sorts interactions by:

  - userId
  - timestamp

**Why sorting is important?**

Because SASRec is a sequential model.
The order of interactions must reflect real user behavior timeline.

If not sorted → sequence becomes meaningless.

In [3]:
PAD_TOKEN = 0
MASK_TOKEN = 1

def load_and_prepare(df):

    df = df.sort_values(["userId", "timestamp"])

    user2seq = defaultdict(list)

    for row in df.itertuples():
        user2seq[row.userId].append(row.movieId)

    # Filter short users
    user2seq = {
        u: seq for u, seq in user2seq.items()
        if len(seq) >= CFG["MIN_INTERACTIONS"]
    }

    # Remap items
    valid_user_ids = set(user2seq.keys())

    item_set = set(
        df[df["userId"].isin(valid_user_ids)]["movieId"].unique()
    )

    item2id = {item: idx+2 for idx, item in enumerate(item_set)}
    id2item = {v: k for k, v in item2id.items()}
    vocab_size = len(item2id) + 2

    remapped_sequences = {
        u: [item2id[i] for i in seq]
        for u, seq in user2seq.items()
    }

    print(f"Total users: {len(remapped_sequences)}")
    print(f"Total items: {len(item2id)}")
    print(f"Vocab size (including PAD & MASK): {vocab_size}")

    return remapped_sequences, vocab_size, item2id, id2item

For each user, If sequence is:
```
[ i1, i2, i3, i4, i5 ]
```
Then:
```
train = [i1, i2, i3]
val   = i4
test  = i5
```
This is called leave-one-out chronological split. Because this is a sequential recommendation problem, random split will leads to data leakage.

In [4]:
def split_sequences(user_sequences):

    train, val, test = {}, {}, {}

    for u, seq in user_sequences.items():
        train[u] = seq[:-2]
        val[u]   = seq[-2]
        test[u]  = seq[-1]

    return train, val, test

## Why we need `sample_negative`?
In implicit-feedback recommendation, we only observe positive interactions
$(u, i^+)$ (e.g., a user watched a movie in MovieLens), but we do not observe explicit negatives. To learn a ranking function $f(u,i)$ that orders items correctly, the model must satisfy $f(u,i^+)>f(u,i^−)$ for items $i^−$ the user did not interact with. However, without negative samples, optimizing only $f(u,i^+)$ leads to a trivial solution where the model increases scores for all items, failing to learn meaningful preferences. By sampling a negative item $𝑖^− ∉ 𝑆_𝑢$ (the user’s interaction set), we can define a pairwise ranking loss such as Bayesian Personalized Ranking (BPR):

$$
𝐿=-∑_{(u, i^+,i^-)} log~σ(f(u,i^+)-f(u,i^−))
$$

which explicitly enforces relative ordering. Thus, negative sampling is mathematically necessary to transform positive-only implicit data into a comparative learning objective that produces a discriminative ranking model.

In [5]:
class SASRecDataset(Dataset):

    def __init__(self, user_sequences, vocab_size):
        self.users = list(user_sequences.keys())
        self.user_sequences = user_sequences
        self.vocab_size = vocab_size

    def sample_negative(self, pos_item, user_seq):
        while True:
            neg = random.randint(2, self.vocab_size-1)
            if neg not in user_seq:
                return neg

    def __getitem__(self, idx):

        user = self.users[idx]
        seq = self.user_sequences[user][-CFG["MAX_LEN"]:]

        padding = CFG["MAX_LEN"] - len(seq)
        seq = [PAD_TOKEN]*padding + seq

        pos_item = seq[-1]
        neg_item = self.sample_negative(pos_item, seq)

        attention_mask = torch.tensor(seq) != PAD_TOKEN

        return {
            "input_ids": torch.tensor(seq),
            "attention_mask": attention_mask.bool(),
            "pos_item": torch.tensor(pos_item),
            "neg_item": torch.tensor(neg_item)
        }

    def __len__(self):
        return len(self.users)

## Model
### High-Level Idea

The model:
- Converts item IDs → embeddings
- Adds positional information
- Passes sequence through Transformer encoder
- Returns representation of last position
- That representation is later used for ranking

So conceptually:

User Sequence → Contextual Representation → Score Items

We get the user embedding from:
```
x[:, -1, :]
```
This is the contextual representation of the entire interaction sequence.

Then score is computed as:
$$
score(u,i)=u^Te_i
$$


where:
- $u$ = final transformer output (dynamic user embedding)
- $e_i$ = item embedding from embedding table (`self.item_emb = nn.Embedding(vocab_size, hidden_dim)`)

In [6]:
class SASRec(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.item_emb = nn.Embedding(vocab_size, CFG["HIDDEN"], padding_idx=PAD_TOKEN)
        self.pos_emb  = nn.Embedding(CFG["MAX_LEN"], CFG["HIDDEN"])

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=CFG["HIDDEN"],
            nhead=CFG["N_HEADS"],
            dim_feedforward=CFG["HIDDEN"]*4,
            dropout=CFG["DROPOUT"],
            batch_first=True
        )

        self.encoder = nn.TransformerEncoder(encoder_layer, CFG["N_LAYERS"])
        self.layer_norm = nn.LayerNorm(CFG["HIDDEN"])
        self.dropout = nn.Dropout(CFG["DROPOUT"])

        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, input_ids, attention_mask):

        positions = torch.arange(0, input_ids.size(1),
                                 device=input_ids.device).unsqueeze(0)

        x = self.item_emb(input_ids) + self.pos_emb(positions)
        x = self.layer_norm(x)
        x = self.dropout(x)

        x = self.encoder(x, src_key_padding_mask=~attention_mask)

        return x[:, -1, :]  # final representation

In [7]:
def bpr_loss(user_repr, pos_emb, neg_emb):

    pos_score = torch.sum(user_repr * pos_emb, dim=1)
    neg_score = torch.sum(user_repr * neg_emb, dim=1)

    return -torch.mean(F.logsigmoid(pos_score - neg_score))

In [8]:
def train(model, dataloader, optimizer):

    model.train()
    scaler = torch.cuda.amp.GradScaler()
    total_loss = 0

    for batch in tqdm(dataloader):

        input_ids = batch["input_ids"].to(CFG["DEVICE"])
        mask = batch["attention_mask"].to(CFG["DEVICE"])
        pos_item = batch["pos_item"].to(CFG["DEVICE"])
        neg_item = batch["neg_item"].to(CFG["DEVICE"])

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():

            user_repr = model(input_ids, mask)

            pos_emb = model.item_emb(pos_item)
            neg_emb = model.item_emb(neg_item)

            loss = bpr_loss(user_repr, pos_emb, neg_emb)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [9]:
def evaluate(
    model,
    train_seq,
    eval_target,
    total_items,
    item_embeddings=None,   # optional for diversity
    k=10,
    neg_samples=100,
    max_len=50,
    pad_token=0,
    device="cuda"
):

    model.eval()

    HR, NDCG, RECALL, MRR, PRECISION = [], [], [], [], []
    all_recommendations = []

    with torch.no_grad():

        for u in train_seq:

            # -----------------------------
            # Prepare sequence
            # -----------------------------
            seq = train_seq[u][-max_len:]
            seq = [pad_token] * (max_len - len(seq)) + seq

            input_ids = torch.tensor(seq).unsqueeze(0).to(device)
            mask = (input_ids != pad_token)  # Proper boolean mask

            # -----------------------------
            # Forward pass
            # -----------------------------
            user_repr = model(input_ids, mask)  # (1, hidden_dim)

            # -----------------------------
            # Candidate sampling
            # -----------------------------
            true_item = eval_target[u]

            candidates = random.sample(
                range(2, model.item_emb.num_embeddings),
                neg_samples
            )

            if true_item not in candidates:
                candidates.append(true_item)

            item_tensor = torch.tensor(candidates).to(device)
            item_embs = model.item_emb(item_tensor)

            # Dot-product scoring
            scores = torch.matmul(user_repr, item_embs.T).squeeze(0)

            # -----------------------------
            # Top-K ranking
            # -----------------------------
            topk_idx = torch.topk(scores, k).indices.cpu().numpy()
            ranked_items = [candidates[i] for i in topk_idx]

            all_recommendations.append(ranked_items)

            # -----------------------------
            # Metrics
            # -----------------------------
            gt = {true_item}

            # HR
            hit = int(true_item in ranked_items)
            HR.append(hit)

            # NDCG
            if hit:
                rank = ranked_items.index(true_item)
                NDCG.append(1 / np.log2(rank + 2))
            else:
                NDCG.append(0.0)

            # Precision@K
            PRECISION.append(hit / k)

            # Recall
            RECALL.append(hit)  # single ground truth → same as HR

            # MRR
            if hit:
                MRR.append(1.0 / (rank + 1))
            else:
                MRR.append(0.0)

    # ---------------------------------
    # Coverage
    # ---------------------------------
    recommended_items = set()
    for ranked in all_recommendations:
        recommended_items.update(ranked)

    coverage = len(recommended_items) / total_items

    # ---------------------------------
    # Diversity (optional)
    # ---------------------------------
    diversity_scores = []

    if item_embeddings is not None:
        for ranked in all_recommendations:

            if len(ranked) < 2:
                diversity_scores.append(0.0)
                continue

            vectors = item_embeddings[ranked]

            total_sim = 0.0
            count = 0

            for i, j in combinations(range(len(vectors)), 2):
                sim = np.dot(vectors[i], vectors[j]) / (
                    norm(vectors[i]) * norm(vectors[j]) + 1e-8
                )
                total_sim += (1 - sim)
                count += 1

            diversity_scores.append(total_sim / count)

        diversity = np.mean(diversity_scores)
    else:
        diversity = None

    # ---------------------------------
    # Final results
    # ---------------------------------
    return {
        "HR@{}".format(k): np.mean(HR),
        "Precision@{}".format(k): np.mean(PRECISION),
        "NDCG@{}".format(k): np.mean(NDCG),
        "Recall@{}".format(k): np.mean(RECALL),
        "MRR@{}".format(k): np.mean(MRR),
        "Coverage@{}".format(k): coverage,
        "Diversity@{}".format(k): diversity
    }


In [10]:
# ================================
# CONFIG
# ================================

DATASET_NAME = "ml-25m"  # change if needed: ml-latest-small
BASE_URL = "https://files.grouplens.org/datasets/movielens/"
DOWNLOAD_URL = f"{BASE_URL}{DATASET_NAME}.zip"
SAVE_PATH = "./data"

# ================================
# STEP 1: Download Dataset
# ================================

def download_movielens():
    os.makedirs(SAVE_PATH, exist_ok=True)
    zip_path = os.path.join(SAVE_PATH, f"{DATASET_NAME}.zip")

    if not os.path.exists(zip_path):
        print("Downloading dataset...")
        response = requests.get(DOWNLOAD_URL)
        with open(zip_path, "wb") as f:
            f.write(response.content)
        print("Download complete.")
    else:
        print("Dataset already downloaded.")

    return zip_path


# ================================
# STEP 2: Extract
# ================================

def extract_dataset(zip_path):
    extract_path = os.path.join(SAVE_PATH, DATASET_NAME)

    if not os.path.exists(extract_path):
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(SAVE_PATH)
        print("Extraction complete.")
    else:
        print("Dataset already extracted.")

    return extract_path


# ================================
# STEP 3: Load ratings.csv
# ================================

def load_ratings(extract_path):
    ratings_path = os.path.join(extract_path, "ratings.csv")

    print("Loading ratings...")
    df = pd.read_csv(ratings_path)

    # Sort by user and timestamp
    df = df.sort_values(["userId", "timestamp"])

    return df


# ================================
# FULL PIPELINE FUNCTION
# ================================

def prepare_movielens():
    zip_path = download_movielens()
    extract_path = extract_dataset(zip_path)
    df = load_ratings(extract_path)

    user_sequences, vocab_size, item2id, id2item = load_and_prepare(df)

    return user_sequences, vocab_size, item2id, id2item

In [11]:
user_sequences, vocab_size, item2id, id2item = prepare_movielens()
train_seq, val_seq, test_seq = split_sequences(user_sequences)

Download complete.
Extracting dataset...
Extraction complete.
Loading ratings...
Total users: 162541
Total items: 59047
Vocab size (including PAD & MASK): 59049


In [ ]:
train_dataset = SASRecDataset(train_seq, vocab_size)
train_loader = DataLoader(train_dataset,
                          batch_size=CFG["BATCH_SIZE"],
                          shuffle=True,
                          num_workers=4)

model = SASRec(vocab_size).to(CFG["DEVICE"])
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["LR"])

for epoch in range(CFG["EPOCHS"]):

    # -----------------------
    # Train
    # -----------------------
    loss = train(model, train_loader, optimizer)

    # -----------------------
    # Evaluate
    # -----------------------
    metrics = evaluate(
        model=model,
        train_seq=train_seq,
        eval_target=val_seq,
        total_items=vocab_size,
        item_embeddings=model.item_emb.weight.detach().cpu().numpy(),
        k=10,
        neg_samples=CFG["NEG_SAMPLES"],
        max_len=CFG["MAX_LEN"],
        pad_token=PAD_TOKEN,
        device=CFG["DEVICE"]
    )

    # -----------------------
    # Logging
    # -----------------------
    print(f"\nEpoch {epoch+1}")
    print(f"Loss: {loss:.4f}")

    print(f"HR@10: {metrics['HR@10']:.4f}")
    print(f"NDCG@10: {metrics['NDCG@10']:.4f}")
    print(f"Recall@10: {metrics['Recall@10']:.4f}")
    print(f"MRR@10: {metrics['MRR@10']:.4f}")
    print(f"Coverage@10: {metrics['Coverage@10']:.4f}")

    if metrics["Diversity@10"] is not None:
        print(f"Diversity@10: {metrics['Diversity@10']:.4f}")



/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-3867157008.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/635 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worke


Epoch 1
Loss: 0.2811
HR@10: 0.9274
NDCG@10: 0.6612
Recall@10: 0.9274
MRR@10: 0.5760
Coverage@10: 0.1900
Diversity@10: 0.4356


100%|██████████| 635/635 [00:16<00:00, 37.93it/s]



Epoch 2
Loss: 0.0871
HR@10: 0.9255
NDCG@10: 0.6559
Recall@10: 0.9255
MRR@10: 0.5698
Coverage@10: 0.3228
Diversity@10: 0.3596


100%|██████████| 635/635 [00:17<00:00, 37.15it/s]


...

100%|██████████| 635/635 [00:17<00:00, 36.25it/s]



Epoch 15
Loss: 0.0001
HR@10: 0.8599
NDCG@10: 0.5295
Recall@10: 0.8599
MRR@10: 0.4270
Coverage@10: 0.2012
Diversity@10: 0.7537


100%|██████████| 635/635 [00:16<00:00, 38.96it/s]



Epoch 16
Loss: 0.0001
HR@10: 0.8376
NDCG@10: 0.5085
Recall@10: 0.8376
MRR@10: 0.4068
Coverage@10: 0.2044
Diversity@10: 0.7697


100%|██████████| 635/635 [00:18<00:00, 35.17it/s]
